In [ ]:
import requests
import multiprocessing
import json

def process_data_locally(data):
    # Your CPU-intensive local function
    result = sum(data)  # Example of CPU-intensive task
    return result

def send_data_to_worker(data_chunk):
    url = "http://192.168.1.63:5000/process"  # Replace worker_pc_ip with the IP of the worker PC
    # Convert data_chunk (range) to a list for JSON serialization
    response = requests.post(url, json={'data': list(data_chunk)})
    if response.status_code == 200:
        return response.json()['result']
    else:
        print("Error communicating with the worker")
        return None


if __name__ == "__main__":
    # Your data to process
    data = range(1, 1000000)

    # Split data into two parts: one for local processing, one for the worker
    data_chunks = [data[:len(data)//2], data[len(data)//2:]]

    # Process part of the data locally
    with multiprocessing.Pool() as pool:
        local_result = pool.apply_async(process_data_locally, (data_chunks[0],))

    # Send the other part to the worker PC
    worker_result = send_data_to_worker(data_chunks[1])

    # Collect the local result
    local_result = local_result.get()

    # Combine results from both local and worker
    total_result = local_result + worker_result

    print("Final Result:", total_result)
